Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [5]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in _notmist.ipynb_.

In [6]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [7]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [8]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [24]:
train_subset = 10000
# LOGISTIC REGRESSION, SGD, L2 regularized, using GD optimzization
log_reg_graph = tf.Graph()
with log_reg_graph.as_default():
    # set constants
    tf_train_data = tf.constant(train_dataset[:train_subset,:])
    tf_train_labels = tf.constant(train_labels[:train_subset,:])
    tf_valid_data = tf.constant(valid_dataset)
    tf_test_data = tf.constant(test_dataset)
    
    # set variables
    tf_W = tf.Variable(tf.truncated_normal([image_size**2, num_labels]))
    tf_b = tf.Variable(tf.zeros([num_labels]))

    # placeholder for regularization constant
    tf_alpha = tf.placeholder_with_default(input=1.0, shape=[])
    
    # define loss
    logits = tf.matmul(tf_train_data, tf_W) + tf_b
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + tf_alpha*tf.nn.l2_loss(tf_W)
    
    # define optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # make predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_data, tf_W) + tf_b)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_data, tf_W) + tf_b)
    
gd_steps = 801
with tf.Session(graph=log_reg_graph) as LRSess:
    # initialize
    tf.initialize_all_variables().run()
    print('Initialized')
    # make some predictions
    for step in range(gd_steps):
        _, l, tr_preds = LRSess.run([optimizer, loss, train_prediction], feed_dict={tf_alpha:.1})
        if step % 100 == 0:
            print('loss at step %s: %s' % (step, l))
            print('training accuracy is %.1f%%' % accuracy(tr_preds, train_labels[:train_subset,:]))
            print('Validation accuracy is %.1f%%' % accuracy(valid_prediction.eval(),valid_labels))
            print('Test accuracy is %.1f%%' % accuracy(test_prediction.eval(), test_labels))
            print('----------------------')
    # 

Initialized
loss at step 0: 329.634
training accuracy is 8.1%
Validation accuracy is 10.8%
Test accuracy is 10.0%
----------------------
loss at step 100: 1.00982
training accuracy is 81.4%
Validation accuracy is 80.9%
Test accuracy is 87.6%
----------------------
loss at step 200: 0.997527
training accuracy is 81.5%
Validation accuracy is 80.9%
Test accuracy is 87.5%
----------------------
loss at step 300: 0.996819
training accuracy is 81.6%
Validation accuracy is 81.0%
Test accuracy is 87.6%
----------------------
loss at step 400: 0.996622
training accuracy is 81.6%
Validation accuracy is 81.0%
Test accuracy is 87.5%
----------------------
loss at step 500: 0.996561
training accuracy is 81.6%
Validation accuracy is 81.0%
Test accuracy is 87.6%
----------------------
loss at step 600: 0.996541
training accuracy is 81.6%
Validation accuracy is 81.0%
Test accuracy is 87.6%
----------------------
loss at step 700: 0.996534
training accuracy is 81.6%
Validation accuracy is 81.0%
Test ac

we were able to improve the validation and test accuracy from Assignment #2

In [57]:
# now let's implement hte L2 regularization for the case of SGD with 2 layer RELU NN

# define graph
my_l2_nn = tf.Graph()
size_hidden_layer = 1024
batch_size = 128

with my_l2_nn.as_default():
    
    # constants
    tf_valid_data  = tf.constant(valid_dataset)
    tf_test_data = tf.constant(test_dataset)
    
    
    # place holders
    tf_train_data = tf.placeholder(dtype = tf.float32, shape=[batch_size, image_size**2])
    tf_train_labels = tf.placeholder(dtype = tf.float32, shape=[batch_size, num_labels])
    tf_alpha = tf.placeholder_with_default(.1,shape=[])
    # variables
    tf_W1 = tf.Variable(tf.truncated_normal([image_size**2, size_hidden_layer]))
    tf_b1 = tf.Variable(tf.truncated_normal([size_hidden_layer]))
    
    tf_W2 = tf.Variable(tf.truncated_normal([size_hidden_layer, num_labels]))
    tf_b2 = tf.Variable(tf.truncated_normal([num_labels]))
    
    # forward prop
    layer1 = tf.matmul(tf_train_data, tf_W1) + tf_b1
    logits = tf.matmul(tf.nn.relu(layer1), tf_W2) + tf_b2
    
        
    # loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + tf_alpha*(tf.nn.l2_loss(tf_W1) + tf.nn.l2_loss(tf_W2)))
  
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # prediction
    predict_train = tf.nn.softmax(logits)
    predict_valid = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_data, tf_W1) + tf_b1), tf_W2) + tf_b2)
    predict_test = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_data, tf_W1) + tf_b1), tf_W2) + tf_b2)
    
    

# now begin a session
n_steps = 16001

with tf.Session(graph=my_l2_nn) as my_l2_nn_sess:
    #initialize
    tf.initialize_all_variables().run()
    print('Initialized')
    # loop over steps
    for step in range(n_steps):
        
        #define mini batch randomly
        i_start = np.random.randint(1000-batch_size)
        # implmeent overfitting
        
        mb_train_data = train_dataset[i_start:batch_size + i_start, :]
        mb_train_labels = train_labels[i_start:i_start + batch_size,:]
        feed_dict = {tf_train_data: mb_train_data, tf_train_labels: mb_train_labels, tf_alpha: .0}
        
        # run
        _, l = my_l2_nn_sess.run([optimizer, loss], feed_dict = feed_dict)
            # evaluate accuracy
        if step % 2000 == 0:
            print('at step %s loss is %s' % (step, l))
            
            print('validation accuracy is %s' % accuracy(predict_valid.eval(),valid_labels))
            print('test accuracy is %s' % accuracy(predict_test.eval(), test_labels))
            print('----------')

Initialized
at step 0 loss is 413.293
validation accuracy is 22.06
test accuracy is 24.14
----------
at step 2000 loss is 0.0
validation accuracy is 77.82
test accuracy is 84.75
----------
at step 4000 loss is 0.0
validation accuracy is 77.81
test accuracy is 84.74
----------
at step 6000 loss is 2.98023e-08
validation accuracy is 77.81
test accuracy is 84.74
----------
at step 8000 loss is 0.0
validation accuracy is 77.81
test accuracy is 84.74
----------
at step 10000 loss is 0.0
validation accuracy is 77.81
test accuracy is 84.74
----------
at step 12000 loss is 0.0
validation accuracy is 77.81
test accuracy is 84.74
----------
at step 14000 loss is 0.0
validation accuracy is 77.81
test accuracy is 84.74
----------
at step 16000 loss is 0.0
validation accuracy is 77.81
test accuracy is 84.74
----------


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [58]:
# now let's implement hte L2 regularization for the case of SGD with 2 layer RELU NN

# define graph
my_l2_nn = tf.Graph()
size_hidden_layer = 1024
batch_size = 128

with my_l2_nn.as_default():
    
    # constants
    tf_valid_data  = tf.constant(valid_dataset)
    tf_test_data = tf.constant(test_dataset)
    
    
    # place holders
    tf_train_data = tf.placeholder(dtype = tf.float32, shape=[batch_size, image_size**2])
    tf_train_labels = tf.placeholder(dtype = tf.float32, shape=[batch_size, num_labels])
    tf_alpha = tf.placeholder_with_default(.1,shape=[])
    # variables
    tf_W1 = tf.Variable(tf.truncated_normal([image_size**2, size_hidden_layer]))
    tf_b1 = tf.Variable(tf.truncated_normal([size_hidden_layer]))
    
    tf_W2 = tf.Variable(tf.truncated_normal([size_hidden_layer, num_labels]))
    tf_b2 = tf.Variable(tf.truncated_normal([num_labels]))
    
    # forward prop
    layer1 = tf.matmul(tf_train_data, tf_W1) + tf_b1
    logits = tf.matmul(tf.nn.relu(layer1), tf_W2) + tf_b2
    
        
    # loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + tf_alpha*(tf.nn.l2_loss(tf_W1) + tf.nn.l2_loss(tf_W2)))
  
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # prediction
    predict_train = tf.nn.softmax(logits)
    predict_valid = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_data, tf_W1) + tf_b1), tf_W2) + tf_b2)
    predict_test = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_data, tf_W1) + tf_b1), tf_W2) + tf_b2)
    
    

# now begin a session
n_steps = 16001

with tf.Session(graph=my_l2_nn) as my_l2_nn_sess:
    #initialize
    tf.initialize_all_variables().run()
    print('Initialized')
    # loop over steps
    for step in range(n_steps):
        
        #define mini batch randomly
        i_start = np.random.randint(1000-batch_size)
        # implmeent overfitting
        i_start = np.random.randint(2)
        mb_train_data = train_dataset[i_start:batch_size + i_start, :]
        mb_train_labels = train_labels[i_start:i_start + batch_size,:]
        feed_dict = {tf_train_data: mb_train_data, tf_train_labels: mb_train_labels, tf_alpha: .0}
        
        # run
        _, l = my_l2_nn_sess.run([optimizer, loss], feed_dict = feed_dict)
            # evaluate accuracy
        if step % 2000 == 0:
            print('at step %s loss is %s' % (step, l))
            
            print('validation accuracy is %s' % accuracy(predict_valid.eval(),valid_labels))
            print('test accuracy is %s' % accuracy(predict_test.eval(), test_labels))
            print('----------')

Initialized
at step 0 loss is 315.853
validation accuracy is 38.45
test accuracy is 41.72
----------
at step 2000 loss is 8.3819e-09
validation accuracy is 66.14
test accuracy is 73.06
----------
at step 4000 loss is 8.3819e-09
validation accuracy is 66.14
test accuracy is 73.06
----------
at step 6000 loss is 8.3819e-09
validation accuracy is 66.14
test accuracy is 73.06
----------
at step 8000 loss is 7.45058e-09
validation accuracy is 66.14
test accuracy is 73.06
----------
at step 10000 loss is 7.45058e-09
validation accuracy is 66.14
test accuracy is 73.06
----------
at step 12000 loss is 7.45058e-09
validation accuracy is 66.14
test accuracy is 73.06
----------
at step 14000 loss is 7.45058e-09
validation accuracy is 66.14
test accuracy is 73.06
----------
at step 16000 loss is 7.45058e-09
validation accuracy is 66.14
test accuracy is 73.06
----------


the validation and testing error top off at 67 and 74 percent respectively while the loss is zero and cant train anymore

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [62]:
# now let's implement hte L2 regularization for the case of SGD with 2 layer RELU NN

# define graph
my_l2_nn = tf.Graph()
size_hidden_layer = 1024
batch_size = 128

with my_l2_nn.as_default():
    
    # constants
    tf_valid_data  = tf.constant(valid_dataset)
    tf_test_data = tf.constant(test_dataset)
    
    
    # place holders
    tf_train_data = tf.placeholder(dtype = tf.float32, shape=[batch_size, image_size**2])
    tf_train_labels = tf.placeholder(dtype = tf.float32, shape=[batch_size, num_labels])
    tf_alpha = tf.placeholder_with_default(.1,shape=[])
    # variables
    tf_W1 = tf.Variable(tf.truncated_normal([image_size**2, size_hidden_layer]))
    tf_b1 = tf.Variable(tf.truncated_normal([size_hidden_layer]))
    
    tf_W2 = tf.Variable(tf.truncated_normal([size_hidden_layer, num_labels]))
    tf_b2 = tf.Variable(tf.truncated_normal([num_labels]))
    
    # forward prop
    layer1 = tf.matmul(tf_train_data, tf_W1) + tf_b1
    drop_out_logits = tf.matmul(tf.nn.dropout(tf.nn.relu(layer1),keep_prob=.5), tf_W2) + tf_b2
    logits = tf.matmul(tf.nn.relu(layer1), tf_W2) + tf_b2
    
    
        
    # loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(drop_out_logits, tf_train_labels) + tf_alpha*(tf.nn.l2_loss(tf_W1) + tf.nn.l2_loss(tf_W2)))
  
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # prediction
    predict_train = tf.nn.softmax(logits)
    predict_valid = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_data, tf_W1) + tf_b1), tf_W2) + tf_b2)
    predict_test = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_data, tf_W1) + tf_b1), tf_W2) + tf_b2)
    
    

# now begin a session
n_steps = 16001

with tf.Session(graph=my_l2_nn) as my_l2_nn_sess:
    #initialize
    tf.initialize_all_variables().run()
    print('Initialized')
    # loop over steps
    for step in range(n_steps):
        
        #define mini batch randomly
        i_start = np.random.randint(1000-batch_size)
        # implmeent overfitting
        i_start = np.random.randint(2)
        mb_train_data = train_dataset[i_start:batch_size + i_start, :]
        mb_train_labels = train_labels[i_start:i_start + batch_size,:]
        feed_dict = {tf_train_data: mb_train_data, tf_train_labels: mb_train_labels, tf_alpha: .05}
        
        # run
        _, l = my_l2_nn_sess.run([optimizer, loss], feed_dict = feed_dict)
            # evaluate accuracy
        if step % 2000 == 0:
            print('at step %s loss is %s' % (step, l))
            
            print('validation accuracy is %s' % accuracy(predict_valid.eval(),valid_labels))
            print('test accuracy is %s' % accuracy(predict_test.eval(), test_labels))
            print('----------')

Initialized
at step 0 loss is 16251.9
validation accuracy is 33.9
test accuracy is 38.39
----------
at step 2000 loss is 0.676352
validation accuracy is 70.22
test accuracy is 77.28
----------
at step 4000 loss is 0.670168
validation accuracy is 70.51
test accuracy is 77.34
----------
at step 6000 loss is 0.676017
validation accuracy is 69.87
test accuracy is 76.51
----------
at step 8000 loss is 0.665286
validation accuracy is 70.32
test accuracy is 77.16
----------
at step 10000 loss is 0.672057
validation accuracy is 70.72
test accuracy is 77.44
----------
at step 12000 loss is 0.649723
validation accuracy is 70.39
test accuracy is 76.94
----------
at step 14000 loss is 0.653338
validation accuracy is 70.18
test accuracy is 77.16
----------
at step 16000 loss is 0.666758
validation accuracy is 70.47
test accuracy is 77.34
----------


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
